In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn.metrics

import pmer

In [ ]:
dataset = pmer.datasets.Dota2Dataset.from_csv('dota2.csv')

In [ ]:
def eval_predictions(predictions):
    master_predictions = [0] * len(dataset)
    master_predictions = sum([np.asarray(preds) for preds in predictions]) / len(predictions)
    accuracy = len([p for p in master_predictions if p > 0.5]) / len(master_predictions)
    log_loss = sklearn.metrics.log_loss([1] * len(master_predictions), master_predictions)
    mse = sklearn.metrics.mean_squared_error([1] * len(master_predictions), master_predictions)
    return accuracy, log_loss, mse

In [ ]:
# Accumulate predictions from weaker raters.
n_raters = 5
p = 0.5

predictions = []
performance = []

for i in range(n_raters):
    ds = dataset.subsample(p)
    rater = pmer.EloRater()
    rater.process_dataset(ds)
    preds = rater.predict(dataset.events)
    predictions.append(preds)
    del rater
    del ds
    performance.append(eval_predictions(predictions))
    print('Processed {}/{} raters'.format(i+1, n_raters))
    
for i in range(n_raters):
    ds = dataset.subsample(p)
    rater = pmer.TrueskillRater()
    rater.process_dataset(ds)
    preds = rater.predict(dataset.events)
    predictions.append(preds)
    del rater
    del ds
    performance.append(eval_predictions(predictions))
    print('Processed {}/{} raters'.format(i+1, n_raters))

In [ ]:
master_predictions = [0] * len(dataset)
master_predictions = sum([np.asarray(preds) for preds in predictions]) / len(predictions)

In [ ]:
accuracy = len([p for p in master_predictions if p > 0.5]) / len(master_predictions)
print('Accuracy = {:.2%}'.format(accuracy))

In [ ]:
log_loss = sklearn.metrics.log_loss([1] * len(master_predictions), master_predictions)
print('Logloss = {:.5}'.format(log_loss))

In [ ]:
mse = sklearn.metrics.mean_squared_error([1] * len(master_predictions), master_predictions)
print('MSE = {:.5}'.format(mse))

In [ ]:
plt.hist(master_predictions, bins=25);

In [ ]:
plt.plot([x[0] for x in performance])